In [23]:
from src.dysregnet import dysregnet
import pandas as pd
import pickle
import numpy as np


Get data

In [24]:
"""
grn_breast = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/dysregnet_gtex/GENIE3_output/linkedList_output_slurm_ gene_tpm_v10_breast_mammary_tissu_filtered .csv")
grn_lung = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/dysregnet_gtex/GENIE3_output/linkedList_output_slurm_ gene_tpm_v10_lung_filtered .csv")

exp_lung = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/dysregnet_gtex/gtex/gene_tpm_v10_lung_filtered.csv")
exp_breast = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/dysregnet_gtex/gtex/gene_tpm_v10_breast_mammary_tissu_filtered.csv")
"""

'\ngrn_breast = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/dysregnet_gtex/GENIE3_output/linkedList_output_slurm_ gene_tpm_v10_breast_mammary_tissu_filtered .csv")\ngrn_lung = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/dysregnet_gtex/GENIE3_output/linkedList_output_slurm_ gene_tpm_v10_lung_filtered .csv")\n\nexp_lung = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/dysregnet_gtex/gtex/gene_tpm_v10_lung_filtered.csv")\nexp_breast = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/dysregnet_gtex/gtex/gene_tpm_v10_breast_mammary_tissu_filtered.csv")\n'

In [25]:
"""
grn_exercise = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/exercise_data/regulatory_network.csv")
exp_exerise = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/exercise_data/LUAD_expression.csv")
meta_exercise = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/exercise_data/LUAD_meta.csv")
"""

'\ngrn_exercise = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/exercise_data/regulatory_network.csv")\nexp_exerise = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/exercise_data/LUAD_expression.csv")\nmeta_exercise = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/exercise_data/LUAD_meta.csv")\n'

In [26]:
import os 
current_dir = os.getcwd()

grn_breast = pd.read_csv(current_dir + "/GENIE3_output/linkedList_output_slurm_gene_tpm_v10_breast_mammary_tissu_filtered.csv")
grn_lung = pd.read_csv(current_dir + "/GENIE3_output/linkedList_output_slurm_gene_tpm_v10_lung_filtered.csv")

exp_lung = pd.read_csv(current_dir + "/gtex/gene_tpm_v10_lung_filtered.csv")
exp_breast = pd.read_csv(current_dir + "/gtex/gene_tpm_v10_breast_mammary_tissu_filtered.csv")

grn_exercise = pd.read_csv(current_dir + "/exercise_data/regulatory_network.csv")
exp_exercise = pd.read_csv(current_dir + "/exercise_data/LUAD_expression.csv")
meta_exercise = pd.read_csv(current_dir + "/exercise_data/LUAD_meta.csv")


In [27]:
"""meta_lung = pd.DataFrame({
    'sample': exp_lung.columns[1:],
    'condition': np.random.choice([0, 1], size=len(exp_lung.columns[1:])),
    'gender' : 'FEMALE'
})
meta_lung"""

"meta_lung = pd.DataFrame({\n    'sample': exp_lung.columns[1:],\n    'condition': np.random.choice([0, 1], size=len(exp_lung.columns[1:])),\n    'gender' : 'FEMALE'\n})\nmeta_lung"

In [28]:
grn_breast.head(2)

,regulatoryGene,targetGene,weight
0,PAX8,PAX8-AS1,0.149235
1,ZFY,EIF1AY,0.101827


In [29]:
grn_breast
meta_breast = pd.DataFrame({
    'sample': exp_breast.columns[1:],
    'condition': np.random.choice([0, 1], size=len(exp_breast.columns[1:])),
    'gender' : 'FEMALE'
})
meta_breast.head(2)

,sample,condition,gender
0,GTEX-1117F-2826-SM-5GZXL,1,FEMALE
1,GTEX-111YS-1926-SM-5GICC,1,FEMALE


In [30]:
meta_lung= pd.DataFrame({
    'sample': exp_lung.columns[1:],
    'condition': np.random.choice([0], size=len(exp_lung.columns[1:])),
    'gender' : 'FEMALE'
})
meta_lung.loc[0, "condition"] = 1
meta_lung.head(5)

,sample,condition,gender
0,GTEX-111CU-0326-SM-5GZXO,1,FEMALE
1,GTEX-111FC-1126-SM-5GZWU,0,FEMALE
2,GTEX-111VG-0726-SM-5GIDC,0,FEMALE
3,GTEX-111YS-0626-SM-5GZXV,0,FEMALE
4,GTEX-1122O-0126-SM-5GICA,0,FEMALE


In [31]:
# Step 1: Filter rows where condition is 0 (controls)
controls = meta_breast[meta_breast['condition'] == 0]

# Step 2: Randomly select two control samples
selected_controls = controls.sample(n=2, random_state=42)  # random_state for reproducibility

# Step 3: Update the meta_breast DataFrame to only include the selected controls and all other rows where condition != 0
meta_breast_filtered = pd.concat([
    selected_controls,
    meta_breast[meta_breast['condition'] != 0]
], ignore_index=True)

# Display the updated meta_breast DataFrame
print(meta_breast_filtered)


                       sample  condition  gender
0     GTEX-ZDXO-0126-SM-5S2ND          0  FEMALE
1     GTEX-W5X1-2326-SM-3GIL6          0  FEMALE
2    GTEX-1117F-2826-SM-5GZXL          1  FEMALE
3    GTEX-111YS-1926-SM-5GICC          1  FEMALE
4    GTEX-117XS-1926-SM-5GICO          1  FEMALE
..                        ...        ...     ...
249   GTEX-ZY6K-1626-SM-5GZWV          1  FEMALE
250   GTEX-ZYFC-0826-SM-5E44K          1  FEMALE
251   GTEX-ZYW4-0826-SM-5GIDG          1  FEMALE
252   GTEX-ZYY3-2526-SM-GMXAZ          1  FEMALE
253   GTEX-ZZPU-0626-SM-5E43T          1  FEMALE

[254 rows x 3 columns]


In [32]:
grn_lung.head(2)

,regulatoryGene,targetGene,weight
0,ZFY,TTTY14,0.269712
1,ZFY,EIF1AY,0.248673


Define the confounding variables or the design matrix

In [33]:
exp_lung = exp_lung.set_index(exp_lung.columns[0])
exp_lung = exp_lung.T 
exp_lung.insert(0, "sample", exp_lung. index)
#assert all(exp_lung.iloc[:, 0].values == meta_lung.iloc[:,0].values)

In [34]:
exp_lung.head(2)

Description,sample,WASH7P,OR4F5,CICP27,WASH9P,MTND1P23,MTND2P28,MTCO1P12,MTCO2P12,MTATP8P1,...,MT-ATP8,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-TE,MT-CYB
GTEX-111CU-0326-SM-5GZXO,GTEX-111CU-0326-SM-5GZXO,1.73487,0.015715,0.404045,3.44562,13.60320,701.845,36.4011,7.73125,21.4650,...,20639.6,30316.0,19533.3,9311.24,11979.60,25032.7,9814.89,13675.40,41.73750,17064.5
GTEX-111FC-1126-SM-5GZWU,GTEX-111FC-1126-SM-5GZWU,6.03857,0.023205,0.105289,10.74130,8.65547,327.442,22.0995,8.86811,15.4709,...,12900.5,20179.0,15523.2,9339.42,5804.09,15797.2,2139.15,1751.61,2.64138,11498.2


In [35]:
exp_breast = exp_breast.set_index(exp_breast.columns[0])
exp_breast = exp_breast.T 
exp_breast.insert(0, "sample", exp_breast. index)
#assert all(exp_breast.iloc[:, 0].values == meta_breast.iloc[:,0].values)

In [36]:
exp_breast.head(2)

Description,sample,WASH7P,OR4F5,WASH9P,MTND1P23,MTND2P28,MTCO1P12,MTCO2P12,MTATP8P1,MTATP6P1,...,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-TE,MT-CYB,MT-TP
GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2826-SM-5GZXL,9.70327,0.018298,7.16661,18.5431,1017.12,38.0448,10.6094,20.5299,4373.92,...,41003.6,51020.1,14202.3,14019.8,35322.9,2929.78,1756.72,2.08274,20157.3,2.113370
GTEX-111YS-1926-SM-5GICC,GTEX-111YS-1926-SM-5GICC,3.98499,0.080215,5.65219,17.7260,1147.13,57.2182,12.2619,42.2615,4126.39,...,45532.2,34797.2,19903.3,18822.5,45468.6,11130.40,16237.40,46.87020,28117.2,0.617656


In [37]:
exp_exercise = exp_exercise.set_index(exp_exercise.columns[0])
exp_exercise = exp_exercise.T 
exp_exercise.insert(0, "sample", exp_exercise.index)
#assert all(exp_exercise.iloc[:, 0].values == meta_exercise.iloc[:,0].values)

In [38]:
meta_exercise.head(1)

,sample,_PATIENT,cancer type abbreviation,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,...,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction,sample_type_id,sample_type,_primary_disease,condition
0,TCGA-38-4625-11,TCGA-38-4625,LUAD,66.0,FEMALE,WHITE,Stage IB,NaN,Lung Adenocarcinoma,NaN,...,2973.0,0.0,2973.0,0.0,2973.0,NaN,11.0,Solid Tissue Normal,lung adenocarcinoma,0


Run DysRegNet

In [39]:
# define condition column (0 indicated control, 1 indicates case), Column name for the condition in the meta DataFrame.
conCol='condition'

# define categorical confounder columns in meta dataframe, List of categorical variable names. They should match the name of their columns in the meta Dataframe.
CatCov=['gender']  

# define continuous confounder columns in meta dataframe, List of continuous covariates. They should match the name of their columns in the meta Dataframe.
# ConCov=['gender']

In [40]:
"""
data=dysregnet.run(expression_data = exp_exercise.head(60), 
                   GRN= grn_exercise,
                   meta = meta_exercise.head(60),
                   direction_condition=False,
                   normaltest=True,
                   CatCov = CatCov,
                   ConCov = [conCol],
                   R2_threshold=.2)
                   
"""

'\ndata=dysregnet.run(expression_data = exp_exercise.head(60), \n                   GRN= grn_exercise,\n                   meta = meta_exercise.head(60),\n                   direction_condition=False,\n                   normaltest=True,\n                   CatCov = CatCov,\n                   ConCov = [conCol],\n                   R2_threshold=.2)\n                   \n'

In [41]:
"""
data=dysregnet.run(expression_data = exp_breast.head(7), 
                   GRN= "breast",
                   meta = meta_breast.head(7),
                   direction_condition=False,
                   normaltest=True,
                   #CatCov = CatCov,
                   #ConCov = [conCol],
                   R2_threshold=.2)
"""

'\ndata=dysregnet.run(expression_data = exp_breast.head(7), \n                   GRN= "breast",\n                   meta = meta_breast.head(7),\n                   direction_condition=False,\n                   normaltest=True,\n                   #CatCov = CatCov,\n                   #ConCov = [conCol],\n                   R2_threshold=.2)\n'

In [50]:
#lll 42 Minuten
#lbl 
data=dysregnet.run(expression_data = exp_lung, 
                   GRN= "breast",
                   meta = meta_lung,
                   direction_condition=False,
                   normaltest=True,
                   #CatCov = CatCov,
                   #ConCov = [conCol],
                   R2_threshold=.2)

Processing breast GRN file...
GRN file and model ZIP already exist locally. Skipping download.
Extracting breast model ZIP...
Extracted models/breast_models.zip to models
You did not input any covariates in CatCov or ConCov parameters, proceeding without them.


Processing edges: 90567it [19:52, 75.96it/s]


Ratio of found models:  1.0
Skipped models:  1.0
edges:  {'patient id': ['GTEX-111CU-0326-SM-5GZXO', 'GTEX-111FC-1126-SM-5GZWU', 'GTEX-111VG-0726-SM-5GIDC', 'GTEX-111YS-0626-SM-5GZXV', 'GTEX-1122O-0126-SM-5GICA', 'GTEX-1128S-0726-SM-5N9D6', 'GTEX-117YW-0526-SM-5H11C', 'GTEX-117YX-1326-SM-5H125', 'GTEX-11DXX-0626-SM-5Q5AG', 'GTEX-11DXZ-0726-SM-5N9C4', 'GTEX-11DYG-0626-SM-GINAE', 'GTEX-11DZ1-0426-SM-5H11A', 'GTEX-11EI6-0826-SM-5985V', 'GTEX-11EMC-0126-SM-5EGKV', 'GTEX-11EQ9-0226-SM-5A5JX', 'GTEX-11GSP-0726-SM-5986L', 'GTEX-11I78-0126-SM-5HL6F', 'GTEX-11LCK-0426-SM-5A5M8', 'GTEX-11NSD-0326-SM-5A5LS', 'GTEX-11NUK-0826-SM-5HL4U', 'GTEX-11NV4-1126-SM-5HL6J', 'GTEX-11O72-1326-SM-5BC5A', 'GTEX-11OF3-1126-SM-5986C', 'GTEX-11P7K-0326-SM-59871', 'GTEX-11P81-0226-SM-5HL5M', 'GTEX-11PRG-0926-SM-5EGI8', 'GTEX-11TT1-1626-SM-5EQL7', 'GTEX-11TUW-0526-SM-5LU9A', 'GTEX-11UD2-0726-SM-5EQ69', 'GTEX-11WQC-0626-SM-5EQMF', 'GTEX-11WQK-1226-SM-5GU5Z', 'GTEX-11XUK-0326-SM-HAV26', 'GTEX-11ZTS-1226-SM-5EQMQ', 'GT

In [43]:
data.get_results().head(2)

""
patient id
GTEX-111CU-0326-SM-5GZXO
GTEX-111FC-1126-SM-5GZWU


In [44]:
# Load the pickle file
with open("pickle_models/AHR_FOS.pkl", "rb") as file:
    loaded_model = pickle.load(file)

In [45]:
print("Attributes and methods of the loaded object:")
print(dir(loaded_model))

Attributes and methods of the loaded object:
['HC0_se', 'HC1_se', 'HC2_se', 'HC3_se', '_HCCM', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abat_diagonal', '_cache', '_data_attr', '_data_in_cache', '_get_robustcov_results', '_get_wald_nonlinear', '_is_nested', '_transform_predict_exog', '_use_t', '_wexog_singular_values', 'aic', 'bic', 'bse', 'centered_tss', 'compare_f_test', 'compare_lm_test', 'compare_lr_test', 'condition_number', 'conf_int', 'conf_int_el', 'cov_HC0', 'cov_HC1', 'cov_HC2', 'cov_HC3', 'cov_kwds', 'cov_params', 'cov_type', 'df_model', 'df_resid', 'eigenvals', 'el_test', 'ess', 'f_pvalue', 'f_test', 'fittedvalues', 'fvalue', 'get_influence', 'get_prediction',

In [46]:
print("Variables and their values in the loaded object:")
print(vars(loaded_model))

Variables and their values in the loaded object:
{'_results': <statsmodels.regression.linear_model.OLSResults object at 0x16aaa2fd0>, '__doc__': "\n    Results class for for an OLS model.\n\n    Parameters\n    ----------\n    model : RegressionModel\n        The regression model instance.\n    params : ndarray\n        The estimated parameters.\n    normalized_cov_params : ndarray\n        The normalized covariance parameters.\n    scale : float\n        The estimated scale of the residuals.\n    cov_type : str\n        The covariance estimator used in the results.\n    cov_kwds : dict\n        Additional keywords used in the covariance specification.\n    use_t : bool\n        Flag indicating to use the Student's t in inference.\n    **kwargs\n        Additional keyword arguments used to initialize the results.\n\n    See Also\n    --------\n    RegressionResults\n        Results store for WLS and GLW models.\n\n    Notes\n    -----\n    Most of the methods and attributes are inherit

In [47]:
print(loaded_model.params)  # Coefficients
print(loaded_model.summary())  # Full model summary

const          0.000360
AHR            0.323355
gender_MALE   -0.000850
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     3.268
Date:                Tue, 28 Jan 2025   Prob (F-statistic):             0.0454
Time:                        17:28:36   Log-Likelihood:                -80.461
No. Observations:                  59   AIC:                             166.9
Df Residuals:                      56   BIC:                             173.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------

In [48]:
# get the patient-specific dysregulate networks
# results = data.get_results()

# or with binary edges
results = data.get_results_binary()

# get R2 values, coefficients, and coefficient p-values for all models/edges
# data.get_model_stats()

In [49]:
edge_counts = results.sum()
most_frequent_edge = edge_counts.idxmax()
max_count = edge_counts.max()

print (f"The most frequent edge is {most_frequent_edge} with {max_count} occurrences.")

ValueError: attempt to get argmax of an empty sequence